## Дообучение ruBERT

Весь следующий код был выполнен в Kaggle Notebook на GPU T4 x2, обучение на 5 эпохах заняло около
10 часов.  

Если нужно загрузить модель, то скачайте в папку models папку по ссылке https://drive.google.com/drive/folders/1-yWsVN7jqjISkW157CojpK3h_84nHuMk?usp=sharing и напишите, к примеру:  

```python
tokenizer_name = "DeepPavlov/rubert-base-cased"
tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
model_name = "../models/rubert_model"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
```

Необходимые импорты

In [1]:
import pandas as pd
import numpy as np

import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

Чтение данных и удаление лишнего столбца

In [2]:
train_path =  "/kaggle/input/raw-data/raw_train.csv"
test_path = "/kaggle/input/raw-data/raw_test.csv"
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [3]:
train.drop("Unnamed: 0", axis=1, inplace=True)
test.drop("Unnamed: 0", axis=1, inplace=True)

Использование GPU

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Используется устройство:", device)
else:
    device = torch.device("cpu")
    print("Используется устройство:", device)

Создание класса для корректного представления данных

In [5]:
class RubertDataset(Dataset):
    """
    Класс, определяющий методы для корректной трансформации сырых батчей формата DataFrame
    в тензоры для последующей передачи в ruBERT
    """
    def __init__(self, dataframe, tokenizer):
        self.dataframe = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text']
        label = self.dataframe.iloc[idx]['sentiment']

        encoding = self.tokenizer.encode_plus(
            text,
            max_length=400,
            padding='max_length', # Дополнение коротких текстов "пустыми" токенами
            truncation=True,      # Отсечение лишних токенов для длинных текстов
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),           # Идентификаторы токенов
            'attention_mask': encoding['attention_mask'].flatten(), # Маска для токенов, которые надо обработать
            'labels': torch.tensor(label, dtype=torch.long)          # Метки классов
        }

Загрузка токенизатора и модели

In [6]:
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)
for param in model.parameters(): param.data = param.data.contiguous() # надо было для корректного сохранения модели

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Определение параметров для обучения

In [7]:
train_dataset = RubertDataset(train, tokenizer)

training_args = TrainingArguments(
    output_dir = '/kaggle/working/models/rubert_model',
    per_device_train_batch_size=50 ,                 # Размер батча
    num_train_epochs=5,                              # Количество эпох
    logging_steps=50,                                # Частота вывода логов
    save_steps=1500,                                 # Частота сохранения модели
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

Обучение

In [8]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
50,0.828800
100,0.613900
150,0.563300
200,0.538500
250,0.523300
300,0.504900
350,0.504900
400,0.519200
450,0.480400
500,0.484100


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=9495, training_loss=0.27381803754632755, metrics={'train_runtime': 39699.7018, 'train_samples_per_second': 23.916, 'train_steps_per_second': 0.239, 'total_flos': 1.95167460960108e+17, 'train_loss': 0.27381803754632755, 'epoch': 5.0})

Сохранение модели в output

In [9]:
trainer.save_model('/kaggle/working/models/rubert_model/final')

Функция для оценки метрик

In [12]:
def evaluate_model(y_test, y_pred) -> pd.DataFrame:
    """
    Оценка метрик accuracy, precision, recall, f1-score на каждом классе с последующим усреднением
    :param y_test: тестовые таргеты
    :param y_pred: предсказанные таргеты
    :return: 
    """

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    metrics = {
        'Метрика': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Значение': [accuracy, precision, recall, f1]
    }

    df_metrics = pd.DataFrame(metrics)

    return df_metrics

Предсказания и оценка на тестовых данных

In [13]:
test_dataset = RubertDataset(test, tokenizer)
predictions = trainer.predict(test_dataset)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [14]:
y_pred = predictions.predictions.argmax(axis=1)
evaluate_model(test.sentiment, y_pred)

,Метрика,Значение
0,Accuracy,0.807944
1,Precision,0.799779
2,Recall,0.800572
3,F1 Score,0.800158


Пример работы с рандомными текстами

In [53]:
example = {
           "text": 
                [
                    "Прекрасный врач! В следующий раз обязательно запишусь к нему снова!",
                    "Ужасный врач! В следующий раз к нему не приду!",
                    "Нормальный врач. Возможно, приду еще. Не уверен"
                ],
           "sentiment": 
                [
                    1,
                    2,
                    0
                ]
          }
example_df = pd.DataFrame.from_dict(example)
example_dataset = RubertDataset(example_df, tokenizer)
trainer.predict(example_dataset).predictions.argmax(axis=1)

array([1, 2, 0])